There are minor changes compared to the code provided in lectures. Main difference - 2 models for 2 genders (male and female). At the end there is code for streamlit app, but github link is also provided below.

[Github link](https://github.com/lturcinskas/namegenerator/tree/main)
[Streamlit app](https://namegenerator-2t7dpe5h8mvkpzlh8gqqrx.streamlit.app/)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [3]:
import requests
from bs4 import BeautifulSoup

names_man = []
names_woman = []
for key in ['a', 'b', 'c', 'c-2', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
            'm', 'n', 'o', 'p', 'r', 's', 's-2', 't', 'u', 'v', 'z', 'z-2']:
    url_man = f'https://vardai.vlkk.lt/sarasas/{key}/?lytis=vyro&kilme='
    url_woman = f'https://vardai.vlkk.lt/sarasas/{key}/?lytis=moters&kilme='
    response_man = requests.get(url_man)
    response_woman = requests.get(url_woman)

    soup_man = BeautifulSoup(response_man.text, 'html.parser')
    links_man = soup_man.find_all('a', class_='names_list__links names_list__links--man')
    names_man += [name.text for name in links_man]

    soup_woman = BeautifulSoup(response_woman.text, 'html.parser')
    links_woman = soup_woman.find_all('a', class_='names_list__links names_list__links--woman')
    names_woman += [name.text for name in links_woman]

In [4]:
import unicodedata

In [5]:
def normalize_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove accentuation
    name = unicodedata.normalize('NFD', name)
    name = ''.join(char for char in name if unicodedata.category(char) != 'Mn')
    return name

In [6]:
names_man = [normalize_name(name) for name in names_man]
names_woman = [normalize_name(name) for name in names_woman]

In [7]:
names_woman += ' '
names_man += ' '

names_man.insert(0, 'name') #arba su np.savetxt headeri name uzdet
names_woman.insert(0, 'name')


In [8]:
np.savetxt("names_man.txt", names_man, delimiter=" ", fmt="%s")
np.savetxt("names_woman.txt", names_woman, delimiter=" ", fmt="%s")

In [9]:
class NameDataset(Dataset):
    def __init__(self, file):
        self.names = pd.read_csv(file)['name'].values
        self.chars = sorted(list(set(''.join(self.names) + ' ')))  # Including a padding character
        self.char_to_int = {c: i for i, c in enumerate(self.chars)}
        self.int_to_char = {i: c for c, i in self.char_to_int.items()}
        self.vocab_size = len(self.chars)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx] + ' '  # Adding padding character at the end
        encoded_name = [self.char_to_int[char] for char in name]
        return torch.tensor(encoded_name)

In [12]:
# Custom collate function for padding
def pad_collate(batch):
    padded_seqs = pad_sequence(batch, batch_first=True, padding_value=0)
    input_seq = padded_seqs[:, :-1]
    target_seq = padded_seqs[:, 1:]
    return input_seq, target_seq

# Minimal Transformer Model
class MinimalTransformer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, forward_expansion):
        super(MinimalTransformer, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(torch.randn(1, 100, embed_size))
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3) #cia didinti bloku skaiciu? 1
        self.output_layer = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        positions = torch.arange(0, x.size(1)).unsqueeze(0)
        x = self.embed(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.output_layer(x)
        return x

# Training Loop
def train_model(model, dataloader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(epochs):
        model.train()  # Ensure the model is in training mode
        total_loss = 0.0
        batch_count = 0

        for batch_idx, (input_seq, target_seq) in enumerate(dataloader):
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()
            output = model(input_seq)
            loss = criterion(output.transpose(1, 2), target_seq)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

        average_loss = total_loss / batch_count
        print(f'Epoch {epoch+1}, Average Loss: {average_loss}')



In [13]:
dataset_man = NameDataset('names_man.txt')
dataloader_man = DataLoader(dataset_man, batch_size=64, shuffle=True, collate_fn=pad_collate)
model_man = MinimalTransformer(vocab_size=dataset_man.vocab_size, embed_size=128, num_heads=8, forward_expansion=4).to(device)
train_model(model_man, dataloader_man)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Average Loss: 1.3387996349178377
Epoch 2, Average Loss: 1.1272727378079148
Epoch 3, Average Loss: 1.1099656595558416
Epoch 4, Average Loss: 1.0915091389515361
Epoch 5, Average Loss: 1.088947838447133
Epoch 6, Average Loss: 1.0832314100421843
Epoch 7, Average Loss: 1.0788561672460837
Epoch 8, Average Loss: 1.0753047085199199
Epoch 9, Average Loss: 1.067140616354395
Epoch 10, Average Loss: 1.0625188936952685


In [14]:
dataset_woman = NameDataset('names_woman.txt')
dataloader_woman = DataLoader(dataset_woman, batch_size=64, shuffle=True, collate_fn=pad_collate)
model_woman = MinimalTransformer(vocab_size=dataset_woman.vocab_size, embed_size=128, num_heads=8, forward_expansion=4).to(device)
train_model(model_woman, dataloader_woman)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Average Loss: 1.3395247530581347
Epoch 2, Average Loss: 1.155941805732784
Epoch 3, Average Loss: 1.1396119994903677
Epoch 4, Average Loss: 1.1288281714738304
Epoch 5, Average Loss: 1.1247657003687388
Epoch 6, Average Loss: 1.1121744931633792
Epoch 7, Average Loss: 1.116031784619858
Epoch 8, Average Loss: 1.1142886877059937
Epoch 9, Average Loss: 1.106497514603743
Epoch 10, Average Loss: 1.092913765515854


In [15]:
def sample(model, dataset, start_str='a', max_length=20, temperature=1.0):
    assert temperature > 0, "Temperature must be greater than 0"
    model.eval()  # Switch model to evaluation mode
    with torch.no_grad():
        # Convert start string to tensor
        chars = [dataset.char_to_int[c] for c in start_str]
        input_seq = torch.tensor(chars).unsqueeze(0)  # Add batch dimension

        output_name = start_str
        for _ in range(max_length - len(start_str)):
            output = model(input_seq)

            # Apply temperature scaling
            logits = output[0, -1] / temperature
            probabilities = torch.softmax(logits, dim=0)

            # Sample a character from the probability distribution
            next_char_idx = torch.multinomial(probabilities, 1).item()
            next_char = dataset.int_to_char[next_char_idx]

            if next_char == ' ':  # Assume ' ' is your end-of-sequence character
                break

            output_name += next_char
            # Update the input sequence for the next iteration
            input_seq = torch.cat([input_seq, torch.tensor([[next_char_idx]])], dim=1)

        return output_name

In [ ]:
# After training your model, generate a name starting with a specific letter
print('More confident male names:')
for _ in range(10):
    print(' ', sample(model_man, dataset_man, start_str='r', temperature=0.5))  # More confident

print('\nMore diverse/creative:')
for _ in range(10):
    print(' ', sample(model_man, dataset_man, start_str='r', temperature=1.5))  # More diverse

More confident male names:
  rentas
  raris
  ralgartas
  rauris
  rerius
  renas
  rarvas
  rilis
  ralelijus
  raris

More diverse/creative:
  ryfettinas
  reonas
  raben
  rargitas
  rormes
  rortis
  raekenrdiks
  rotovaldv
  reimelo
  rij


In [ ]:
print('More confident female names:')
for _ in range(10):
    print(' ', sample(model_woman, dataset_woman, start_str='r', temperature=0.5))  # More confident

print('\nMore diverse/creative:')
for _ in range(10):
    print(' ', sample(model_woman, dataset_woman, start_str='r', temperature=1.5))  # More diverse

More confident female names:
  riste
  rara
  ralitaninonta
  railarija
  rale
  rala
  rarilija
  raja
  ralija
  retalija

More diverse/creative:
  rana
  relfre
  reln
  rofidza
  raba
  rudesijara
  rapriode
  resmezina
  rirllimija
  rausaud


In [16]:
train_model(model_woman, dataloader_woman, epochs=200)

Epoch 1, Average Loss: 1.114335157978001
Epoch 2, Average Loss: 1.108429463052038
Epoch 3, Average Loss: 1.1107363033650526
Epoch 4, Average Loss: 1.0939897530114473
Epoch 5, Average Loss: 1.0930923205703051
Epoch 6, Average Loss: 1.0981490558652736
Epoch 7, Average Loss: 1.0958489909100888
Epoch 8, Average Loss: 1.0999143657399648
Epoch 9, Average Loss: 1.0929918547174824
Epoch 10, Average Loss: 1.0901802440187824
Epoch 11, Average Loss: 1.0977277444369757
Epoch 12, Average Loss: 1.0931045955686427
Epoch 13, Average Loss: 1.085070425894723
Epoch 14, Average Loss: 1.0845106235191004
Epoch 15, Average Loss: 1.0790811163275988
Epoch 16, Average Loss: 1.075735516512572
Epoch 17, Average Loss: 1.0858380305233286
Epoch 18, Average Loss: 1.0897123724666995
Epoch 19, Average Loss: 1.0938931828114524
Epoch 20, Average Loss: 1.0984405768451406
Epoch 21, Average Loss: 1.0817569645483103
Epoch 22, Average Loss: 1.0853762697817675
Epoch 23, Average Loss: 1.0733654641393404
Epoch 24, Average Loss: 

In [17]:
train_model(model_man, dataloader_man, epochs=200)

Epoch 1, Average Loss: 1.074590512963592
Epoch 2, Average Loss: 1.0763142744048697
Epoch 3, Average Loss: 1.0720172006575788
Epoch 4, Average Loss: 1.0668733686697287
Epoch 5, Average Loss: 1.059357090074508
Epoch 6, Average Loss: 1.0640636731366642
Epoch 7, Average Loss: 1.0618759686829613
Epoch 8, Average Loss: 1.0598775334045536
Epoch 9, Average Loss: 1.0564299460317268
Epoch 10, Average Loss: 1.0570586079456767
Epoch 11, Average Loss: 1.0589039618851708
Epoch 12, Average Loss: 1.0571346312272745
Epoch 13, Average Loss: 1.056117994863479
Epoch 14, Average Loss: 1.0554364216132242
Epoch 15, Average Loss: 1.0522273651889114
Epoch 16, Average Loss: 1.0576558054470626
Epoch 17, Average Loss: 1.0561159868709376
Epoch 18, Average Loss: 1.0506356211959338
Epoch 19, Average Loss: 1.0536832819219495
Epoch 20, Average Loss: 1.0446469295220298
Epoch 21, Average Loss: 1.0654459400255172
Epoch 22, Average Loss: 1.0533289586911436
Epoch 23, Average Loss: 1.0400346419850335
Epoch 24, Average Loss:

In [18]:
torch.save(model_man, 'model_man.pt')
torch.save(model_woman, 'model_woman.pt')

Below is the code for streamlitapp.

In [ ]:
import streamlit as st
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas as pd


class NameDataset(Dataset):
    def __init__(self, file):
        self.names = pd.read_csv(file)['name'].values
        self.chars = sorted(list(set(''.join(self.names) + ' ')))  # Including a padding character
        self.char_to_int = {c: i for i, c in enumerate(self.chars)}
        self.int_to_char = {i: c for c, i in self.char_to_int.items()}
        self.vocab_size = len(self.chars)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx] + ' '  # Adding padding character at the end
        encoded_name = [self.char_to_int[char] for char in name]
        return torch.tensor(encoded_name)


class MinimalTransformer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, forward_expansion):
        super(MinimalTransformer, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(torch.randn(1, 100, embed_size))
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3) #cia didinti bloku skaiciu? 1
        self.output_layer = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        positions = torch.arange(0, x.size(1)).unsqueeze(0)
        x = self.embed(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.output_layer(x)
        return x


# Load your PyTorch models
# Replace with your actual model paths and loading logic
model_man = torch.load('model_man.pt', map_location=torch.device('cpu'))
model_woman = torch.load('model_woman.pt', map_location=torch.device('cpu'))

dataset_man = NameDataset('names_man.txt')
dataset_woman = NameDataset('names_woman.txt')


def sample(model, dataset, start_str='a', max_length=20, temperature=1.0):
    assert temperature > 0, "Temperature must be greater than 0"
    model.eval()  # Switch model to evaluation mode
    with torch.no_grad():
        # Convert start string to tensor
        chars = [dataset.char_to_int[c] for c in start_str]
        input_seq = torch.tensor(chars).unsqueeze(0)  # Add batch dimension

        output_name = start_str
        for _ in range(max_length - len(start_str)):
            output = model(input_seq)

            # Apply temperature scaling
            logits = output[0, -1] / temperature
            probabilities = torch.softmax(logits, dim=0)

            # Sample a character from the probability distribution
            next_char_idx = torch.multinomial(probabilities, 1).item()
            next_char = dataset.int_to_char[next_char_idx]

            if next_char == ' ':  # Assume ' ' is your end-of-sequence character
                break

            output_name += next_char
            # Update the input sequence for the next iteration
            input_seq = torch.cat([input_seq, torch.tensor([[next_char_idx]])], dim=1)

        return output_name


# Streamlit App
st.title("Lithuanian name Generator")

# Input text
input_text = st.text_input("Enter starting letters:", "")

# Model selection
model_option = st.radio("Select gender:", ("Male", "Female"))


temperature = st.slider(
    label="Adjust the temperature (higher temperature - more \"creative\" names)",
    min_value=0.1,  # Minimum value
    max_value=2.0,  # Maximum value
    value=1.0,      # Default value
    step=0.1        # Step size
)


# Generate button
if st.button("Generate name"):
    if not input_text:
        st.error("Please enter some starting letters.")
    else:
        # Select the appropriate model
        selected_model = model_man if model_option == "Male" else model_woman
        selected_dataset = dataset_man if model_option == "Female" else dataset_woman

        result = sample(selected_model, selected_dataset, start_str=input_text.lower(), temperature=temperature)
        print(result)
        st.success(f"Generated name: {result}")